In [43]:
!pip install datasets
!pip install --upgrade accelerate
!pip install transformers[torch]
!pip install evaluate
!pip install sacrebleu

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [44]:
!pip install gradio

In [45]:
from datasets import Dataset
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import accelerate
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import EncoderDecoderModel
from transformers import AutoTokenizer

In [46]:
from datasets import load_dataset

ds = load_dataset("phuong123/finetuned_term_ev")
ds

DatasetDict({
    train: Dataset({
        features: ['en', 'vi', 'src', 'tar'],
        num_rows: 15088
    })
    val: Dataset({
        features: ['en', 'vi', 'src', 'tar'],
        num_rows: 3234
    })
    test: Dataset({
        features: ['en', 'vi', 'src', 'tar'],
        num_rows: 3233
    })
})

In [47]:
from transformers import T5Tokenizer

model_name = "t5-base"  # or "t5-base", "t5-large", etc.
tokenizer = T5Tokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    inputs = ["translate English to Vietnamese: " + ex for ex in examples["en"]]
    targets = examples["vi"]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_ds = ds.map(preprocess_function, batched=True)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [48]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_name)


In [49]:
from transformers import T5ForConditionalGeneration
model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [50]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
)


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [51]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)


In [52]:
import wandb
wandb.login(key="fdc7d2e2f3c7033f00cdc0f0b54d120446be006c")
wandb.init()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁█
train/global_step,▁█
train/grad_norm,▁
train/learning_rate,▁
train/loss,▁
eval/loss,0.55484
eval/runtime,16.6496


In [53]:
trainer.train()


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,0.554838
2,1.306900,0.490614
3,0.579000,0.458714
4,0.530600,0.436087
5,0.503900,0.419806
6,0.479500,0.408027
7,0.468600,0.399822
8,0.457600,0.394187
9,0.452600,0.391370
10,0.446000,0.390316


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=4720, training_loss=0.5742190975253865, metrics={'train_runtime': 2040.4608, 'train_samples_per_second': 73.944, 'train_steps_per_second': 2.313, 'total_flos': 5105092751523840.0, 'train_loss': 0.5742190975253865, 'epoch': 10.0})

In [54]:
metrics = trainer.evaluate(tokenized_ds["test"])
print(metrics)


{'eval_loss': 0.3804484009742737, 'eval_runtime': 16.6029, 'eval_samples_per_second': 194.725, 'eval_steps_per_second': 6.144, 'epoch': 10.0}


In [55]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
def translate(text):
    input_text = "translate English to Vietnamese: " + text
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    outputs = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text

# Test the translation function
test_sentence = "Dear God, thank you for granting us the evergreen garden of this world"
translated_sentence = translate(test_sentence)
print(f"Translated sentence: {translated_sentence}")

Translated sentence: Các bnh tnh khác


In [56]:
import gradio as gr

def translate(text):
    input_text = "translate English to Vietnamese: " + text
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    outputs = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text

iface = gr.Interface(
    fn=translate,
    inputs="text",
    outputs="text",
    title="English to Vietnamese Translation",
    description="Translate English text to Vietnamese using a fine-tuned T5 model."
)


In [57]:
iface.launch(share = True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://cacf9d6b0324379ea3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
